In [ ]:
# Install all required packages
%package install bloomberg.bquant.signal_lab=1.5.2

In [ ]:
import bloomberg.bquant.signal_lab as signal_lab
signal_lab.__version__

In [5]:
import bql

from bloomberg.bquant.signal_lab.workflow.node import (
    industry_grouping, portfolio_construction)
from bloomberg.bquant.signal_lab.signal.transformers import WeightingScheme
from bloomberg.bquant.signal_lab.workflow.factory import (
    UniverseFactory,
    DataItemFactory,
    SignalFactory,
)
from bloomberg.bquant.signal_lab.workflow import (
    AnalyticsDataConfig,
    build_backtest,
)

from bloomberg.bquant.signal_lab.workflow.utils import get_sandbox_path

import utils.event_backtest_helper as ebh
import backtest_params as bp

import numpy as np
import pandas as pd
import importlib

In [ ]:
importlib.reload(ebh)

In [6]:
# Get the saved DataPack path
bq = bql.Service()
data_pack_path = f"{get_sandbox_path()}/esl/datapack_snapshot"
data_pack_path

's3://awmgd-prod-finml-sandbox-user/bclarke16/esl/datapack_snapshot'

In [ ]:
# Key backtest parameters
start = "2020-01-05"
end = "2024-03-01"
universe_name = "INDU Index"

# Trade information
# Import the test data set while developing
df = pd.read_csv('test_analysis.csv')
df = df[['Date','Security','Decision','Confidence']]
df = df.head(1)

In [ ]:
# Get all of the data pack items needed for an events backtest

universe, benchmark, trading_calendar = bp.get_universe_params(start, end, universe_name, data_pack_path)
price, cur_mkt_cap, total_return = bp.get_return_params(start, end, data_pack_path)
analytics_data_config = bp.get_analytics_data_config(start, end, universe_name, data_pack_path)

In [ ]:
# Functions for the conversion of BBG securities to FIGIs for ESL

def _bql_execute_single(univ: list[str], field: dict[str, bql.om.bql_item.BqlItem]) -> pd.DataFrame:
    """Execute a BQL query with one field"""
    req = bql.Request(univ, field)
    data = bq.execute(req)
    return data[0].df()

def convert_to_figi(df: pd.DataFrame) -> pd.DataFrame:
    """Function to convert Bloomberg tickers in a dataframe to FIGIs for ESL"""
    univ      = df['Security'].to_list()
    field     = {'figi': bq.data.composite_id_bb_global()}
    figi      = _bql_execute_single(univ, field)
    merged_df = df.merge(figi, left_on='Security', right_index=True).sort_index()
    return merged_df[['Date', 'figi', 'Decision', 'Confidence']].rename(columns={'figi':'Security'})

df1 = convert_to_figi(df)
df1

In [ ]:
def build_port_weights(signal: pd.DataFrame, events_df: pd.DataFrame) -> pd.DataFrame:
    long_portfolio =  signal.copy(deep=True)
    short_portfolio = signal.copy(deep=True)
    
    long_portfolio.loc[:,:] = False
    short_portfolio.loc[:,:] = False
    signal.loc[:,:] = 1
    
    # STEP 1 get the list of securities in th df_events database
    unique_securities = list(events_df['Security'].unique())
    
    # STEP 2: iterate over the list of securities to look at the individual trades
    for security in unique_securities:
        try:
            security_trades = events_df[events_df['Security'] == security]
        
            # STEP 3: iterate over the trades and update the long/ short portfolio depending on trade direction
            for row in security_trades.itertuples():
                if row.Decision == 'BUY':
                    long_portfolio[security].loc[row.Date:] = True
                    short_portfolio[security].loc[row.Date:] = False
                if row.Decision == 'SELL':
                    long_portfolio[security].loc[row.Date:] = False
                    short_portfolio[security].loc[row.Date:] = True
                if row.Decision == 'HOLD':
                    continue
                if row.Decision == 'Missing':
                    continue
        except KeyError:
            print(f"Missing: {security}")
    
        # STEP 4: create an equal weighted long and short leg
    long_portfolio_leg = ebh.leg_portfolio(
        signal=signal,
        weighting_scheme=WeightingScheme.EQUAL,
        assets_filter=long_portfolio,
        long_leg=True
    )
    
    short_portfolio_leg = ebh.leg_portfolio(
        signal=signal,
        weighting_scheme=WeightingScheme.EQUAL,
        assets_filter=short_portfolio,
        long_leg=False
    )
    
    long_short_portfolio = long_portfolio_leg.add(
        short_portfolio_leg,
        fill_value=0.0,
    )
    
    # STEP 4: return the long and short portfolios
    return long_short_portfolio#long_portfolio_leg, short_portfolio_leg

In [ ]:
def event_backtest(df_events: pd.DataFrame, universe: UniverseFactory, signal: SignalFactory) -> pd.DataFrame:
    """Take a list of trades and create the weights in the portfolio
    df_events: DataFrame with Date, Security, Decision (BUY/ SELL/ HOLD) and Confidence Columns
    universe:  UniverseFactory object from Bloomberg Equity Signal Lab
    signal:    SignalFactory object from Bloomberg Equity Signal Lab - this is usually price
    """
    converted_df = convert_to_figi(df_events)
    
    signal.bind_universe(universe)
    signal_df = signal.df()
    return build_port_weights(signal_df,converted_df)

In [ ]:
lsp = event_backtest(df, universe, price) 

In [ ]:
price.bind_universe(universe)
price_df = price.df()

In [ ]:


port_long_short = portfolio_construction.from_user(
    compute_weights_fn=build_port_weights,
    total_returns=total_return,
    trading_calendar=trading_calendar,
    implementation_lag=1,
    rebalance_freq="D",
    #trading_dates=trading_dates,
    events_df=df1,
    signal=price_df,
)

In [ ]:
def price_signal(signal:  DataItemFactory) -> DataItemFactory:
    return signal

prices_signal = SignalFactory.from_user(
    user_func=price_signal,
    start=start,
    end=end,
    label="market_cap",
    signal=total_return,
)


backtest = build_backtest(
    universe=universe,                                  # Univ of choice from DataPack
    benchmark_universe=benchmark,                       # Benchmark of choice from DataPack 
    start=start,                                        # Backtest start date
    end=end,                                            # Backtest end date
    namespace='events-bt',                 # The user S3 sandbox storage 
    signals=[prices_signal],                                 # My list of signals to use

    portfolio_construction = port_long_short,

    reports=[
        "PerformanceReport",
        "QuantileAnalyticsReport",
        "DescriptiveStatisticsReport",
    ],
    analytics_data_config= analytics_data_config,
)

In [ ]:
backtest.signals

In [ ]:
backtest_results = backtest.evaluate_graph()

In [ ]:
#backtest_results

In [ ]:
backtest_results.plot_analytics()

In [60]:
import event_study as es
import importlib
importlib.reload(es)

<module 'event_study' from '/project/event_study.py'>

In [61]:
# Key backtest parameters
start = "2020-01-05"
end = "2024-03-01"
universe_name = "INDU Index"


backtest = es.EventBacktest(start,end,'INDU Index',data_pack_path)

100%|██████████| 1/1 [00:05<00:00,  5.99s/it]


In [62]:
df = pd.read_csv('test_analysis.csv')
df = df[['Date','Security','Decision','Confidence']]
df = df.head(1)


In [63]:
#df['Decision'] = 'SELL'
df

,Date,Security,Decision,Confidence
0,2020-02-06,MMM UN Equity,BUY,70.0


In [64]:
bt = backtest.run(df,'test1')
bt.plot_analytics()

100%|██████████| 117/117 [00:32<00:00,  3.63it/s]


In [65]:
df_strategy, df_benchmark = backtest.get_return_data()

In [66]:
df_strategy

,Cum. Return (Gross),Cum. Return (Net),Cum. T-Cost
DATE,,,
2020-05-01,0.000000,0.000000,-0.0
2020-05-04,0.000000,0.000000,-0.0
2020-05-05,-0.006536,-0.006536,-0.0
2020-05-06,-0.014879,-0.014879,-0.0
2020-05-07,-0.018026,-0.018026,-0.0
...,...,...,...
2024-02-26,-0.179477,-0.179477,-0.0
2024-02-27,-0.174250,-0.174250,-0.0
2024-02-28,-0.183350,-0.183350,-0.0


In [28]:
df['Decision'] = 'SELL'
df

,Date,Security,Decision,Confidence
0,2020-02-06,MMM UN Equity,SELL,70.0


In [29]:
bt = backtest.run(df,'test1')
bt.plot_analytics()


100%|██████████| 117/117 [00:36<00:00,  3.21it/s]


In [34]:
list(bt.analytics.keys())

['(\'PerformanceStatisticsByYear\', "IndexWeights[\'INDU Index\']")',
 '(\'PerformanceStatistics\', "IndexWeights[\'INDU Index\']")',
 '(\'MarketValue\', "IndexWeights[\'INDU Index\']")',
 '(\'Turnover\', "IndexWeights[\'INDU Index\']")',
 '(\'MaxDrawdown\', "IndexWeights[\'INDU Index\']")',
 '(\'SharpeRatio\', "IndexWeights[\'INDU Index\']")',
 '(\'Drawdown\', "IndexWeights[\'INDU Index\']")',
 '(\'Holdings\', "IndexWeights[\'INDU Index\']")',
 '(\'Volatility\', "IndexWeights[\'INDU Index\']")',
 '(\'AnnualizedReturn\', "IndexWeights[\'INDU Index\']")',
 '(\'CumulativeReturn\', "IndexWeights[\'INDU Index\']")',
 '(\'RollingVolatility\', "IndexWeights[\'INDU Index\']")',
 '(\'RollingSharpeRatio\', "IndexWeights[\'INDU Index\']")',
 '(\'Weights\', "IndexWeights[\'INDU Index\']")',
 '(\'TransactionCost\', "IndexWeights[\'INDU Index\']")',
 '(\'Returns\', "IndexWeights[\'INDU Index\']")',
 '(\'FactorWeights\', "IndexWeights[\'INDU Index\']")',
 "('ActivePerformanceStatisticsByYear', 'COMB

In [59]:
# Return time series
df_strategy_return = bt.analytics["CumulativeReturn"].read()['gross']['COMBINED'].read()
# Benchmark Return
df_benchmark_return = bt.analytics["CumulativeReturn"].read()['benchmark']["IndexWeights['INDU Index']"].read()
# Strategy YoY Performance
df_strategy_performance = bt.analytics['PerformanceStatisticsByYear'].read()['gross']['COMBINED'].read()
# Benchmark YoY Performance
df_benchmark_performance = bt.analytics['PerformanceStatisticsByYear'].read()['benchmark']["IndexWeights['INDU Index']"].read()
# Rolling Sharpe Ratio -strategy
df_rolling_sharpe_strategy = bt.analytics['RollingSharpeRatio'].read()['gross']["COMBINED"].read()
# Rolling Sharpe Ratio -benchmark
df_rolling_sharpe_benchmark = bt.analytics['RollingSharpeRatio'].read()['benchmark']["IndexWeights['INDU Index']"].read()


In [58]:
bt.analytics['RollingSharpeRatio'].read()['benchmark']["IndexWeights['INDU Index']"].read()

,Sharpe Ratio (Gross),Sharpe Ratio (Net)
DATE,,
2020-05-01,NaN,NaN
2020-05-04,NaN,NaN
2020-05-05,NaN,NaN
2020-05-06,NaN,NaN
2020-05-07,NaN,NaN
...,...,...
2024-02-26,1.787624,1.787624
2024-02-27,1.868415,1.868415
2024-02-28,1.846398,1.846398
